In [1]:
from tradealgorithm.tradealgorithm import TradeAlgorithm
from tradealgorithm.tradealgorithmfactory import TradeAlgorithmFactory
from pricer.pricer_factory import PricerFactory
from pricer.pricer import Pricer
from ranker.ranker_factory import RankerFactory
from ranker.ranker import Ranker
from classifier.classifier_factory import ClassifierFactory
from classifier.classifier import Classifier
from datetime import datetime
from tqdm import tqdm

In [2]:
trade_algo = TradeAlgorithmFactory.build(TradeAlgorithm.RRR_BETA)

In [3]:
ranker_class = Ranker.NONE
classifier_class = Classifier.NONE
current = False

In [4]:
start = datetime(2001,1,1)
end = datetime(2023,1,1)
current_date = datetime.now()

In [5]:
pricer_classes = [] 
pricer_classes.append(Pricer.DAILY_STOCK_ROLLING)

In [6]:
for pricer_class in tqdm(pricer_classes):
    try:
        trade_algo.initialize(pricer_class,ranker_class,classifier_class,start,end,current_date)
        trade_algo.initialize_bench_and_yields()
        trade_algo.initialize_classes()
        trade_algo.load_optimal_parameter()
        simulation = trade_algo.create_current_simulation()
        returns = trade_algo.create_returns(True)
        merged = trade_algo.merge_sim_returns(simulation,returns)
        complete = trade_algo.apply_yields(merged,True)
        trade_algo.initialize_backtester()
        trade_algo.drop_recommendations()
        trades = trade_algo.run_recommendation(complete)
        trade_algo.db.cloud_connect()
        trade_algo.db.store("recs",trades)
        trade_algo.db.disconnect()
    except Exception as e:
        print(trade_algo.name,str(e))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:50<00:00, 50.50s/it]


In [7]:
complete

,year,quarter,month,week,date,ticker,price_prediction,close,high,low,...,weekly_yield10,monthly_yield10,quarterly_yield10,market_dately_return,projected_dately_return,dately_delta,dately_delta_sign,dately_rrr_tyield1,dately_rrr_tyield2,dately_rrr_tyield10
0,2023,1.0,1.0,1.0,2023-01-03,A,150.879063,150.04,153.1300,148.470,...,1.000725,1.003145,1.009465,1.000383,0.007233,0.007233,1.0,0.000100,0.000088,0.000074
1,2023,1.0,1.0,1.0,2023-01-03,AAL,13.109500,12.74,13.0000,12.525,...,1.000725,1.003145,1.009465,1.000383,0.029003,0.029003,1.0,0.000075,0.000062,0.000047
2,2023,1.0,1.0,1.0,2023-01-03,AAP,142.392709,151.54,151.6599,146.000,...,1.000725,1.003145,1.009465,1.000383,-0.048926,0.048926,-1.0,0.000062,0.000048,0.000032
3,2023,1.0,1.0,1.0,2023-01-03,AAPL,136.010845,125.07,130.9000,124.170,...,1.000725,1.003145,1.009465,1.000383,0.089134,0.089134,1.0,0.000051,0.000036,0.000020
4,2023,1.0,1.0,1.0,2023-01-03,ABBV,160.126288,162.38,163.0200,160.810,...,1.000725,1.003145,1.009465,1.000383,0.004767,0.004767,1.0,0.000112,0.000100,0.000086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17543,2023,2.0,6.0,22.0,2023-06-02,YUM,134.694695,134.53,134.6100,132.780,...,1.000717,1.003113,1.009368,1.000383,0.001224,0.001224,1.0,0.000185,0.000170,0.000154
17544,2023,2.0,6.0,22.0,2023-06-02,ZBH,132.943500,129.81,130.1200,127.360,...,1.000717,1.003113,1.009368,1.000383,0.024139,0.024139,1.0,0.000092,0.000070,0.000047
17545,2023,2.0,6.0,22.0,2023-06-02,ZBRA,270.281000,278.01,279.6100,269.330,...,1.000717,1.003113,1.009368,1.000383,-0.027801,0.027801,-1.0,-0.000033,-0.000065,-0.000098
17546,2023,2.0,6.0,22.0,2023-06-02,ZION,26.259910,29.07,29.5600,28.170,...,1.000717,1.003113,1.009368,1.000383,-0.096666,0.096666,-1.0,-0.000199,-0.000244,-0.000291
